### 1. Import library

In [3]:
#This project use python 3.12.2 enviroment
#The enviroment helpful to analytics with libraries
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


### 2. Read data

In [2]:
#Read the dates 
data = pd.read_csv("speed_data.csv")
data.head()
data = data.dropna()

3501

In [4]:
#Separate target variable
data_y = data[data.columns[5]]
data_y.head()
#All other dates
data_x = data.drop(columns=["dec"])
data_x.head()

NameError: name 'dates' is not defined

##### 2.1 Normalize

In [ ]:
#Standardization dates
dates_x = pd.get_dummies(dates_x)

#Scale values out of range
escalador = StandardScaler()
dates_x = escalador.fit_transform(dates_x)

##### 2.2 Shape 

In [ ]:
#Shape data in train and test
dates_x.shape[0]

X_train, X_test, Y_train,Y_test = train_test_split(dates_x,dates_y,test_size=0.2, random_state=2)

print("X Train: {}, X Test: {}, y_train {}, y_test {}".format(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape))

n_inputs = X_train.shape[1]

X Train: (2800, 217), X Test: (701, 217), y_train (2800,), y_test (701,)


##### 2.3 Transform to tensors

In [ ]:
#transform the array of np or pnds to tensors
t_X_train = torch.from_numpy(X_train).float().to("cuda")
t_X_test = torch.from_numpy(X_test).float().to("cuda")
t_Y_train = torch.from_numpy(Y_train.values).float().to("cuda")
t_Y_test = torch.from_numpy(Y_test.values).float().to("cuda")
t_Y_train = t_Y_train[:,None]
t_Y_test = t_Y_test[:,None]

#define tensor test
test = TensorDataset(t_X_test,t_Y_test)

### 3 Create a CNN

In [ ]:
class Red(nn.Module):
    def __init__(self, n_inputs):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_inputs, 128)
        self.dropout1 = nn.Dropout(0.3)  
        self.linear2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.1)  
        self.linear3 = nn.Linear(64, 32)
        self.linear4 = nn.Linear(32, 1)

    def forward(self, inputs):
        x = F.relu(self.linear1(inputs))
        x = self.dropout1(x) 
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)  
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        return x

##### 3.1 Create a train function and test function

In [ ]:
#Train function
def train(model, optimizer, loss_fn, t_X_train, t_Y_train, epochs, status_print):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        Y_pred = model(t_X_train)
        loss = loss_fn(input=Y_pred, target=t_Y_train)
        loss.backward()
        optimizer.step()

        if epoch % status_print == 0:
            print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")

# Test function
def test(model, t_X_test, t_Y_test, status_print):
    model.eval()
    with torch.no_grad():
        Y_pred = model(t_X_test)
        Y_pred_class = Y_pred.round()
        correct = (Y_pred_class == t_Y_test).sum()
        accuracy = 100 * correct / float(len(t_Y_test))

        if status_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
        return accuracy.item()

##### 3.3 Parameters of function

In [ ]:
lr = 0.001
epochs = 2000
status_print = 100

#3.4 Inicialice model 

In [ ]:
model = Red(n_inputs=n_inputs)
model.to("cuda")
loss_fn = nn.BCELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

### 4 Make a prediction with model

In [ ]:
# Train and test model
print("Entrenando el modelo")
train(model, optimizer, loss_fn, t_X_train, t_Y_train, epochs, status_print)
print("Evaluando el modelo")
test(model, t_X_test, t_Y_test, status_print)

Entrenando el modelo

Epoch 100 	 Loss: 0.0033

Epoch 200 	 Loss: 0.0014

Epoch 300 	 Loss: 0.0052

Epoch 400 	 Loss: 0.0027

Epoch 500 	 Loss: 0.007

Epoch 600 	 Loss: 0.0065

Epoch 700 	 Loss: 0.0009

Epoch 800 	 Loss: 0.0025

Epoch 900 	 Loss: 0.0009

Epoch 1000 	 Loss: 0.0012

Epoch 1100 	 Loss: 0.0015

Epoch 1200 	 Loss: 0.0031

Epoch 1300 	 Loss: 0.002

Epoch 1400 	 Loss: 0.0019

Epoch 1500 	 Loss: 0.0013

Epoch 1600 	 Loss: 0.0019

Epoch 1700 	 Loss: 0.0012

Epoch 1800 	 Loss: 0.0028

Epoch 1900 	 Loss: 0.0058

Epoch 2000 	 Loss: 0.0046
Evaluando el modelo


tensor(83.4522, device='cuda:0')